In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
from requests import get
import time

In [6]:
def get_urls(num_pages):
    pages = list(range(1,num_pages+1))
    page_urls = []
    for page in pages:
        p = f'https://github.com/search?o=desc&p={page}&q=stars%3A%3E1&s=stars&type=Repositories'
        page_urls.append(p)
    repo_links_all = []
    for page in page_urls:
        url = page
        headers  = {'User-Agent' : 'Codeup Data Science Student'} #user-agent to decrease chance of being rejected by github
        response = get(url,headers)
        soup = BeautifulSoup(response.text) #created a BS object
        repo_list = soup.find('ul', class_='repo-list') #find section of text of interest
        links = repo_list.find_all(href = True) #isolate the 'href's 
        repo_links= [link['href'] for link in links if not re.match(r'^/topics',link['href']) and not re.match(r'^https',link['href'])\
                 and link['href'].count('/') < 3] #further pare down list by excluding anything that begins with /topics, https or more than 2 forward slashes
        repo_links_all.extend(repo_links)
        time.sleep(3)
    return repo_links_all

In [13]:
url_list = get_urls(1)

AttributeError: 'NoneType' object has no attribute 'find_all'